In [2]:





#%%


# def read_secret(secret_name):
#     secret_path = os.getenv(secret_name)
#     try:
#         with open(secret_path, 'r') as file:
#             return file.read().strip()
#     except Exception as e:
#         print(f"Error reading {secret_name}: {e}")
#         return None

# INFLUXDB_USERNAME = read_secret('DOCKER_INFLUXDB_INIT_USERNAME_FILE')
# INFLUXDB_PASSWORD = read_secret('DOCKER_INFLUXDB_INIT_PASSWORD_FILE')
# INFLUXDB_TOKEN = read_secret('DOCKER_INFLUXDB_INIT_ADMIN_TOKEN_FILE')

# INFLUXDB_ORG = os.getenv('DOCKER_INFLUXDB_INIT_ORG')
# INFLUXDB_BUCKET = os.getenv('DOCKER_INFLUXDB_INIT_BUCKET')
# INFLUXDB_URL = os.getenv('INFLUXDB_URL', 'http://influxdb:8086') 
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

from influxdb_client.client.query_api import QueryOptions
import pandas as pd
import os

INFLUXDB_USERNAME="admin"
INFLUXDB_PASSWORD="password"
INFLUXDB_TOKEN="G3UtSut5Kv-RuT32yh27StdDCrl4fu3uzxPLzdias8vFsNzyzgfw5kIX9iGvtLctAXpZjFItOUwA65YWtk_5fg=="
INFLUXDB_ORG="example_org"
INFLUXDB_BUCKET="example_bucket"
INFLUXDB_URL="http://localhost:8086"





In [7]:
from datetime import datetime, timedelta, timezone
import time
import pytz

class AnomalyDetector():
    def __init__(self,url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG,bucket=INFLUXDB_BUCKET):
        self.url = url
        self.token=token
        self.org = org
        self.bucket=bucket
        self.client = InfluxDBClient(url=self.url, token=self.token, org=self.org)
        self.query_api = self.client.query_api(query_options=QueryOptions(profilers=[]))
        
        self.write_api = self.client.write_api(write_options=SYNCHRONOUS)
        self.training_df = pd.DataFrame()
        self.upcoming_df = pd.DataFrame()
        self.bucket=bucket
        self.last_pull_time = datetime.now()
    def run(self):
        self.get_dataframe_initial()
        self.train_model()
        while True:
            new_df = self.get_dataframe_starting(self.last_pull_time)
            # HERE TEST THIS NEW DATA FOR ANOMALY DETECTION.
            if new_df.empty:
                time.sleep(5)
                continue
            
            # DETECT ANOMALIES
            anomaly_flagged_df = self.detect_anomalies(new_df)
            self.push_data_to_influxdb(anomaly_flagged_df)
            
            # # FOR DEBUGGING ONLY
            # only_anomaly = (anomaly_flagged_df[anomaly_flagged_df.anomaly])
            # if only_anomaly.empty:
            #     time.sleep(5)
            #     continue
            # print(only_anomaly)
            
            # ADD NON-ANOMALY DATA TO BUFFER FOR TRAINIG
            self.extend_non_anomaly_df(anomaly_flagged_df)
            
            # IF BUFFER HAS ONE YEAR, TRAIN NEW DATA   
            if self.has_one_year(self.upcoming_df):
                self.training_df = self.upcoming_df
                self.upcoming_df = pd.DataFrame()
            
            
            time.sleep(5)
    def push_data_to_influxdb(self,df):

        points = [
            Point("anomaly_data")
            .tag("unit", "bpm")
            .field("noisy", row['noisy'])
            .field("anomaly", row['anomaly'])
            .field("original_time", row['original_time'].isoformat())  # Storing original time as a field
            .time(datetime.utcnow(),WritePrecision.MS)  # Use current UTC time for timestamp
            for _, row in df.iterrows()
        ]
        
        self.write_api.write(bucket=self.bucket, record=points)
            
    def has_one_year(self, df):
        time_span = df['original_time'].max() - df['original_time'].min()
        return time_span >= pd.Timedelta(days=365)
    
    def extend_non_anomaly_df(self,flagged_df):
        #get non_anomaly data
        non_anomaly_data= flagged_df[~flagged_df.anomaly][['noisy','original_time']]
        
        # extend future training data
        self.upcoming_df = pd.concat([self.upcoming_df,non_anomaly_data])
    
    def train_model(self):
        pass
    
    def detect_anomalies(self, new_df):
        # Simple Anomaly Detection (Just for testing the class)
        new_df['threshold_anomaly'] = new_df['noisy'] > 70
        
        
        # Final decision for classification
        new_df['anomaly'] = new_df['threshold_anomaly'] 
        return new_df
    
    def get_dataframe_starting(self, starting_time):

        query = f'''
        from(bucket: "{INFLUXDB_BUCKET}")
                |> range(start: {starting_time})
                |> filter(fn: (r) => r._measurement == "heart_rate")
                |> filter(fn: (r) => r._field == "noisy" or r._field == "original_time")
                |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
                |> keep(columns: ["noisy", "original_time"])
        '''
        print(query)
        df = self.query_api.query_data_frame(query=query)
        if not df.empty:
            self.last_pull_time = (datetime.utcnow().replace(tzinfo=pytz.utc) -timedelta(seconds=10)).isoformat()
            df.drop(columns=['table','result'],inplace=True)
            df['original_time'] = pd.to_datetime(df['original_time'])
            if not self.upcoming_df.empty:
                mask = ~df['original_time'].isin(self.upcoming_df['original_time'])
                df = df[mask]
        return  df
    
    def get_dataframe_initial(self):
        checkpoint_df = self.get_dataframe_starting('-24h')
        latest_time = checkpoint_df['original_time'].max()
        one_year_before_latest = latest_time - pd.DateOffset(years=1)
        training_df = checkpoint_df[checkpoint_df['original_time'] >= one_year_before_latest]
        training_df['anomaly'] = False
        self.push_data_to_influxdb(training_df)
        self.training_df = training_df


ad = AnomalyDetector(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG,bucket=INFLUXDB_BUCKET)
#ad.run()
#ad.client.close()



In [8]:
ad.last_pull_time.isoformat()

'2024-09-15T16:05:42.776830'

In [9]:
ad.get_dataframe_initial()


        from(bucket: "example_bucket")
                |> range(start: -24h)
                |> filter(fn: (r) => r._measurement == "heart_rate")
                |> filter(fn: (r) => r._field == "noisy" or r._field == "original_time")
                |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
                |> keep(columns: ["noisy", "original_time"])
        


/tmp/ipykernel_4150407/2606115452.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df['anomaly'] = False


In [12]:
tdf= ad.training_df.set_index('original_time').copy()
# Define the window size for the rolling calculations
window_size = '7D'  # Example for daily rolling window if data is hourly

# Calculate rolling mean and standard deviation
tdf['rolling_mean'] = tdf['noisy'].rolling(window=window_size).mean()
tdf['rolling_std'] = tdf['noisy'].rolling(window=window_size).std()
tdf['rolling_Q1'] = tdf['noisy'].rolling(window=window_size).quantile(0.25)
tdf['rolling_Q3'] = tdf['noisy'].rolling(window=window_size).quantile(0.75)
tdf['rolling_IQR'] = tdf['rolling_Q3'] - tdf['rolling_Q1']
tdf.dropna()

,noisy,anomaly,rolling_mean,rolling_std,rolling_Q1,rolling_Q3,rolling_IQR
original_time,,,,,,,
2016-05-26 11:59:32+00:00,65.723971,False,65.631838,0.130295,65.585772,65.677905,0.092133
2016-05-27 11:59:32+00:00,65.890695,False,65.718124,0.175568,65.631838,65.807333,0.175495
2016-05-28 11:59:32+00:00,65.035382,False,65.547439,0.370248,65.413625,65.765652,0.352027
2016-05-29 11:59:32+00:00,66.481153,False,65.734181,0.526476,65.539705,65.890695,0.350990
2016-05-30 11:59:32+00:00,70.460331,False,66.521873,1.986074,65.585772,66.333539,0.747767
...,...,...,...,...,...,...,...
2017-05-23 18:00:00+00:00,66.889781,False,73.729311,26.329177,66.565439,67.189153,0.623714
2017-05-24 09:00:00+00:00,67.711627,False,73.733448,26.328186,66.565439,67.189153,0.623714
2017-05-24 12:00:00+00:00,66.619458,False,73.720807,26.331637,66.565439,67.189153,0.623714


In [ ]:
# Normalize the year for seasonal overlap

tdf.index = pd.to_datetime(tdf.index)
def convert_to_month_day(df):
    # Format the index to 'month-day', ensuring the index is datetime
    month_day_index = df.index.strftime('%m-%d')  # Convert datetime index to 'month-day' string
    df.index = month_day_index  # Set this new index
    return df
tdf = convert_to_month_day(tdf.copy())

In [22]:
new_df = ad.get_dataframe_starting("-20s")

new_df.index = pd.to_datetime(new_df.index)


# Function to convert index to 'month-day' format

# Apply this function to both dataframes
new_df = convert_to_month_day(new_df.copy())

new_df['original_time'] = pd.to_datetime(new_df['original_time'])
new_df.set_index('original_time', inplace=True)
std_multiplier = 3  # Define how many standard deviations away to flag as anomaly

new_df['rolling_mean'] = new_df['noisy'].rolling(window='7D').mean()
new_df['rolling_std'] = new_df['noisy'].rolling(window='7D').std()
new_df['rolling_std_old'] = tdf['rolling_std'].reindex(new_df.index, method='nearest')
new_df['rolling_mean_old'] = tdf['rolling_mean'].reindex(new_df.index, method='nearest')

std_threshold = 2 
new_df['exceeds_threshold'] = new_df['rolling_std'] > std_threshold *new_df['rolling_std_old']

def identify_anomalies(row):
    if row['exceeds_threshold']:
        # Flag as anomaly if the deviation from the mean is unusually large
        return abs(row['noisy'] - row['rolling_mean']) > 1.7 * row['rolling_std']
    return False

# Apply the function
new_df['anomaly'] = new_df.apply(identify_anomalies, axis=1)
cutoff_date = new_df.index.min() + pd.Timedelta(days=7)
new_df = new_df[new_df.index > cutoff_date]
plot_df = new_df.copy()
plot_tdf = tdf.copy()
plot_df.index = plot_df.index.map(lambda x: x.replace(year=2000))
plot_tdf.index = plot_tdf.index.map(lambda x: x.replace(year=2000))
anomalies = plot_df[plot_df['anomaly']]

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

# Prepare the figure with three subplots horizontally
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# First subplot: Plot all noisy data and highlight anomalies
ax1.plot(new_df.index, new_df['noisy'], label='Noisy Data', color='blue', linestyle='--', marker='o', markersize=5, alpha=0.7)
anomalies = new_df[new_df['anomaly'] == True]
ax1.scatter(anomalies.index, anomalies['noisy'], color='red', label='Anomalies', marker='o', s=50)
ax1.set_title('Noisy Data with Anomalies Highlighted')
ax1.set_xlabel('Date (MM-DD)')
ax1.set_ylabel('Noisy Values')
ax1.legend()
ax1.grid(True)

# Second subplot: Historical mean vs. Current mean
ax2.plot(new_df.index, new_df['rolling_mean'], label='Current Rolling Mean', color='green', linestyle='-', linewidth=2)
ax2.plot(tdf.index, tdf['rolling_mean'], label='Historical Rolling Mean', color='purple', linestyle=':', linewidth=2)
ax2.set_title('Rolling Means Comparison')
ax2.set_xlabel('Date (MM-DD)')
ax2.set_ylabel('Mean Values')
ax2.legend()
ax2.grid(True)

# Third subplot: Historical std vs. Current std
ax3.plot(new_df.index, new_df['rolling_std'], label='Current Rolling Std', color='orange', linestyle='-', linewidth=2)
ax3.plot(tdf.index, tdf['rolling_std'], label='Historical Rolling Std', color='cyan', linestyle=':', linewidth=2)
ax3.set_title('Rolling Standard Deviations Comparison')
ax3.set_xlabel('Date (MM-DD)')
ax3.set_ylabel('Standard Deviation')
ax3.legend()
ax3.grid(True)

# Improve formatting of the x-axis dates
for ax in [ax1, ax2, ax3]:
    plt.sca(ax)
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

# Display the plot
plt.show()


if not anomalies.empty:
    print("Detailed Anomaly Information:")
    for idx, row in anomalies.iterrows():
        print(f"Time: {idx}")
        print(f"  Anomaly Noisy Value: {row['noisy']}")
        print(f"  Rolling Mean at Time of Anomaly: {row['rolling_mean']}")
        print(f"  Rolling Standard Deviation at Time of Anomaly: {row['rolling_std']}")
        print(f"  Old Rolling Standard Deviation from Training Data: {row[ 'rolling_std_old']}")
        print("------")
else:
    print("No anomalies detected within the specified parameters.")




        from(bucket: "example_bucket")
                |> range(start: -20s)
                |> filter(fn: (r) => r._measurement == "heart_rate")
                |> filter(fn: (r) => r._field == "noisy" or r._field == "original_time")
                |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
                |> keep(columns: ["noisy", "original_time"])
        


ValueError: cannot reindex on an axis with duplicate labels

In [4]:
#%%
query = f'''
from(bucket: "{INFLUXDB_BUCKET}")
        |> range(start: -5h)
        |> filter(fn: (r) => r._measurement == "heart_rate")
        |> filter(fn: (r) => r._field == "noisy" or r._field == "original_time")
        |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
        |> keep(columns: ["noisy", "original_time"])
'''
checkpoint_df = query_api.query_data_frame(query=query)
client.close()
checkpoint_df.drop(columns=['table','result'],inplace=True)
checkpoint_df['original_time'] = pd.to_datetime(checkpoint_df['original_time'])
latest_time = checkpoint_df['original_time'].max()
one_year_before_latest = latest_time - pd.DateOffset(years=1)
training_df = checkpoint_df[checkpoint_df['original_time'] >= one_year_before_latest]

training_df.head()

# HERE TRAIN DATA USING TRAINING DF
# trainanomalymodel(training_df)

,noisy,original_time
12033,70.090403,2025-05-07 18:00:00+00:00
12034,70.301699,2025-05-08 09:00:00+00:00
12035,69.996953,2025-05-08 12:00:00+00:00
12036,71.090618,2025-05-08 15:00:00+00:00
12037,70.555684,2025-05-08 18:00:00+00:00
